In [102]:
import urllib
import xmltodict
import json
import pandas as pd
from datetime import datetime

In [103]:
today = datetime.today().strftime("%Y%m%d")  # 현재날짜


In [104]:
service_key = 'du7sxfdZ1IgPVwgcIN89XSOUVEow0Eho%2Bv%2B%2BapFpI7P0eHcZazptxafwktGn2BaT3zWsvCy8TE5MNS9suXGZ2g%3D%3D'

base_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
pageNo = '1'
numOfRows = '500'
startCreateDt = "20200101"
endCreateDt = today
request_url = base_url + '?' + "serviceKey=" + service_key+ '&pageNo=' + pageNo + '&numOfRows=' + numOfRows + '&startCreateDt=' + startCreateDt + '&endCreateDt=' + endCreateDt 
print(request_url)


response = urllib.request.urlopen(request_url) #url부름
json_str = response.read().decode("utf-8") #json으로 바꾸기 전에 utf-8로 디코딩

response.status
data_dict=xmltodict.parse(json_str) #딕셔너리로바꾸기


http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=du7sxfdZ1IgPVwgcIN89XSOUVEow0Eho%2Bv%2B%2BapFpI7P0eHcZazptxafwktGn2BaT3zWsvCy8TE5MNS9suXGZ2g%3D%3D&pageNo=1&numOfRows=500&startCreateDt=20200101&endCreateDt=20200927


In [105]:
json_data = json.dumps(data_dict,ensure_ascii = False) #딕셔너리를 제이슨으로
corona_json=json.loads(json_data) #제이슨 읽기
result = pd.DataFrame() #결과값 저장할 데이터 프레임 만들기

for item in corona_json['response']['body']["items"]['item']:
    s = pd.Series([item["stateDt"],int(item["decideCnt"]),int(item["deathCnt"])] ,index = ["기준일","확진자","사망자"])
    result = result.append(s, ignore_index = True)
result #데이터프레임으로 변환한 결과값

,기준일,사망자,확진자
0,20200927,401.0,23611.0
1,20200926,399.0,23516.0
2,20200925,395.0,23455.0
3,20200924,393.0,23341.0
4,20200923,388.0,23216.0
...,...,...,...
269,20200205,0.0,18.0
270,20200204,0.0,0.0
271,20200203,0.0,15.0
272,20200202,2.0,2.0


In [106]:
#누적확진자 데이터에서 일일확진자데이터 만들기
daily_case = []

for i in range(len(corona_json['response']['body']["items"]['item'])):
    try :
        diff = int(result["확진자"][i]) - int(result["확진자"][i+1])
        daily_case.append(diff)
    except :
        daily_case.append(None)
        
result["일일확진자"] = daily_case

In [107]:
#누적사망자 데이터에서 일일확진자데이터 만들기
daily_death = []

for i in range(len(corona_json['response']['body']["items"]['item'])):
    try :
        diff = int(result["사망자"][i]) - int(result["사망자"][i+1])
        daily_death.append(diff)
    except :
        daily_death.append(None)
        
result["일일사망자"] = daily_death

In [108]:
result.tail(5)

,기준일,사망자,확진자,일일확진자,일일사망자
269,20200205,0.0,18.0,18.0,0.0
270,20200204,0.0,0.0,-15.0,0.0
271,20200203,0.0,15.0,13.0,-2.0
272,20200202,2.0,2.0,2.0,2.0
273,20200101,0.0,0.0,NaN,NaN


In [109]:
result.info() #기준일 데이터타입확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준일     274 non-null    object 
 1   사망자     274 non-null    float64
 2   확진자     274 non-null    float64
 3   일일확진자   273 non-null    float64
 4   일일사망자   273 non-null    float64
dtypes: float64(4), object(1)
memory usage: 10.8+ KB


In [110]:
#기준일 데이터타입 날짜데이터로 바꾸기
#1234-56-78형태로바꾸기
new_date = []
for i in result["기준일"] :
    new_date.append(datetime.strptime(str(i), "%Y%m%d").date())
result["기준일"] = new_date

In [111]:
result.head(5)

,기준일,사망자,확진자,일일확진자,일일사망자
0,2020-09-27,401.0,23611.0,95.0,2.0
1,2020-09-26,399.0,23516.0,61.0,4.0
2,2020-09-25,395.0,23455.0,114.0,2.0
3,2020-09-24,393.0,23341.0,125.0,5.0
4,2020-09-23,388.0,23216.0,110.0,0.0


In [112]:
#날짜데이터로 바꾸기
result["기준일"] = pd.to_datetime(result["기준일"])

In [113]:
result.head(5)

,기준일,사망자,확진자,일일확진자,일일사망자
0,2020-09-27,401.0,23611.0,95.0,2.0
1,2020-09-26,399.0,23516.0,61.0,4.0
2,2020-09-25,395.0,23455.0,114.0,2.0
3,2020-09-24,393.0,23341.0,125.0,5.0
4,2020-09-23,388.0,23216.0,110.0,0.0


In [114]:
#인덱스로 만들기
result = result.set_index("기준일", inplace =False)

In [115]:
result.tail(5)

,사망자,확진자,일일확진자,일일사망자
기준일,,,,
2020-02-05,0.0,18.0,18.0,0.0
2020-02-04,0.0,0.0,-15.0,0.0
2020-02-03,0.0,15.0,13.0,-2.0
2020-02-02,2.0,2.0,2.0,2.0
2020-01-01,0.0,0.0,NaN,NaN


In [116]:
#결측치값있는 행 지우기
result = result.dropna(axis = 0)

In [117]:
result

,사망자,확진자,일일확진자,일일사망자
기준일,,,,
2020-09-27,401.0,23611.0,95.0,2.0
2020-09-26,399.0,23516.0,61.0,4.0
2020-09-25,395.0,23455.0,114.0,2.0
2020-09-24,393.0,23341.0,125.0,5.0
2020-09-23,388.0,23216.0,110.0,0.0
...,...,...,...,...
2020-02-05,0.0,19.0,1.0,0.0
2020-02-05,0.0,18.0,18.0,0.0
2020-02-04,0.0,0.0,-15.0,0.0


In [118]:
#일별 사망자, 확진자 파일 저장
result.to_excel("daily_corona_result.xlsx" , index = False, encoding= "euc-kr")

In [119]:
#월별 평균 데이터 만들기
m_corona = result.resample('M').mean()

In [120]:
m_corona

,사망자,확진자,일일확진자,일일사망자
기준일,,,,
2020-02-29,2.771930,408.771930,55.263158,0.298246
2020-03-31,82.636364,7740.787879,201.090909,4.393939
2020-04-30,217.393939,10571.333333,29.666667,2.575758
2020-05-31,260.677419,11039.677419,22.677419,0.741935
2020-06-30,277.533333,12148.600000,44.400000,0.400000
2020-07-31,291.548387,13610.806452,48.548387,0.612903
2020-08-31,307.548387,16116.225806,182.000000,0.741935
2020-09-30,361.592593,22158.925926,135.703704,2.851852


In [81]:
#월별 데이터 저장
m_corona.to_excel("m_corona.xlsx" , index = False, encoding= "euc-kr")